# Разведочный анализ данных 

In [211]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

In [212]:
df = pd.read_csv('C:/Users/User/OneDrive/Documents/car_price_prediction.csv')

df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


Представленный набор данных содержит информацию о различных характеристиках крабов:
1. Car_Name - название машины 
2. Year - год
3. Selling_Price - цена на продаже
4. Present_Price  - текущая стоимость
5. Kms_Driven - пробег
6. Fuel_Type - тип двигателя
7. Seller_Type - тип продавца
8. Transmission	- коробка передач
9. Owner - владелец

Цель работы заключается в том, чтобы предсказать стоимость машины (Present_Price) по каким-то косвенным признакам, известным о ней. Для этого лучше всего использовать задачу регрессии и такие метрики, как: 
- MAE 
- R-squared 

Обоснование выбора: 
- MAE легко интерпретировать, так как данная мера представляет собой среднее абсолютное отклонение предсказанных значений от фактических. Это дает понимание реальной ошибки в тех же единицах, что и сама зависимая переменная. MAE менее чувствительна к выбросам, потому что она не усиливает влияние больших отклонений относительно малых отклонений.
- R-squared показывает какая доля вариативности зависимой переменной объяснима моделью. Значение  равное 1 указывает на идеальное соответствие, а значение 0 указывает на то, что модель совсем не способна объяснить вариативность данных. Метрика полезна для понимания процентного объема вариативности.

# Работа с данными

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


В наборе представлены данные разных типов:
- float64 (2)
- int64 (3)
- object (4)

In [214]:
df.isna().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

Можно увидеть, что пропуски в данных отсутствуют.

In [215]:
df.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


In [216]:
unique_car_names = df['Car_Name'].nunique()
unique_car_names

98

В датасете много уникальных названий машин, поэтому можно избавиться от этого столбца, чтобы не "засорять" данные.

In [217]:
df_dropped = df.drop('Car_Name', axis=1)

df_dropped.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


Столбец 'Year' не несет в себе большого количества полезной информации. Можно создать новый столбец 'Current_age', в котором будет посчитан текущий возраст машины.

In [218]:
df_dropped['Current_age'] = 2024 - df_dropped['Year']
df_dropped = df_dropped.drop('Year', axis=1)

df_dropped.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Current_age
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


# Числовые признаки

In [219]:
num_cols = ['Selling_Price', 'Present_Price', 'Kms_Driven', 'Owner', 'Current_age']

In [220]:
file_names = []

for col in num_cols:
    fig = px.box(df_dropped, y=col, title=f'График ящик с усами для {col}')
    fig.update_layout(height=600, width=800)
    fig.show()
    
    file_name = f'boxplot_{col}.html'
    file_names.append(file_name)
    
    fig.write_html(file_name)

# Категориальные признаки

In [195]:
cat_cols = ['Fuel_Type', 'Seller_Type', 'Transmission']

In [221]:
file_names = []

for col in cat_cols:
    fig = px.pie(df_dropped, names=col, title=f'Процентное соотношение {col}')
    fig.show()
    
    file_name = f'piechart_{col}.html'
    file_names.append(file_name)
    
    fig.write_html(file_name)

- Fuel_Type: наиболее распространены машины, использующие бензин (79.4%)
- Seller_Type: чуть больше половины машин продается диллерами (64.8%)
- Transmission: подавляющие большинство машин имеют механическую коробку передач (86.7%)

Преобразуем категориальные данные в числовые:

In [222]:
df_dummies=pd.get_dummies(df_dropped, drop_first=True)

df_dummies.head()

,Selling_Price,Present_Price,Kms_Driven,Owner,Current_age,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Individual,Transmission_Manual
0,3.35,5.59,27000,0,10,False,True,False,True
1,4.75,9.54,43000,0,11,True,False,False,True
2,7.25,9.85,6900,0,7,False,True,False,True
3,2.85,4.15,5200,0,13,False,True,False,True
4,4.60,6.87,42450,0,10,True,False,False,True


In [223]:
corrmap = df_dummies.corr()

top_corr_features = corrmap.index

fig1 = px.imshow(corrmap,
                labels=dict(color="Correlation"),
                x=top_corr_features,
                y=top_corr_features,
                color_continuous_scale="RdYlGn",
                color_continuous_midpoint=0)

fig1.show()

fig1.write_html("correlation_map.html")

Наибольшую корреляцию между собой имеют признаки Selling_Price и Present_Price (0.8789825). Также можно увидеть взаимосвязь между Kms_Driven и Current_age (0.524342), очевидно, что чем машина старше, тем больше ее пробег. 

# Обучение модели

In [224]:
X = df_dummies.drop("Selling_Price", axis = 1)

y = df_dummies['Selling_Price']

In [225]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

basic_model = LinearRegression()

basic_model.fit(x_train, y_train)

LinearRegression()

In [226]:
y_pred = linr.predict(x_test)

In [227]:
mae = mean_absolute_error(y_test, y_pred)

print('MAE:', mae)

MAE: 1.294498741010437


Целевая переменная - цена текущая стоимость автомобиля. MAE может интерпретироваться как средняя ошибка предсказания в размере примерно 1,294. Иными словами, в среднем предсказанные значения отклоняются от фактических на примерно 1,294 единицы.

In [228]:
print(r2_score(y_test, y_pred))

0.8436773164235075


Коэффициент детерминации стремится к единице, следовательно, модель хорошо объясняет вариацию зависимой переменной.

In [229]:
df_scatter = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

fig2 = px.scatter(df_scatter, x='y_test', y='y_pred', labels={'y_test': 'Истинное значение', 'y_pred': 'Предсказанное значение'})

fig2.add_trace(go.Scatter(x=df_scatter['y_test'], y=df_scatter['y_test'], mode='lines', name='Идельные предсказания'))

fig2.update_layout(title='График рассеяния', xaxis_title='Истинные значения', yaxis_title='Предсказанные значения')
fig2.show()

fig2.write_html("scatter_plot.html")

В большинстве своем точки на графике лежат близко к линии, проходящей через начало координат, что указывает на хорошее качество модели.